## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-13-20-53-36 +0000,nypost,Greenland’s prime minister: ‘We choose Denmark...,https://nypost.com/2026/01/13/us-news/greenlan...
1,2026-01-13-20-51-30 +0000,cbc,"As Carney heads to China to talk trade, Ontari...",https://www.cbc.ca/news/canada/toronto/doug-fo...
2,2026-01-13-20-51-00 +0000,wsj,The Pentagon said it would invest $1 billion i...,https://www.wsj.com/politics/national-security...
3,2026-01-13-20-48-54 +0000,nyt,Trump Urges Antigovernment Protesters in Iran ...,https://www.nytimes.com/2026/01/13/world/middl...
4,2026-01-13-20-48-07 +0000,nyt,"In a Risky Gambit, Trump Tries Brute Force to ...",https://www.nytimes.com/2026/01/13/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2427/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,62
35,iran,29
408,fed,20
263,ice,16
90,powell,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
179,2026-01-13-14-47-00 +0000,wsj,Some Trump administration officials and allies...,https://www.wsj.com/politics/policy/trump-alli...,148
375,2026-01-12-22-04-31 +0000,cbc,Bank of Canada governor defends U.S. Fed's Pow...,https://www.cbc.ca/news/politics/bank-of-canad...,137
205,2026-01-13-13-38-58 +0000,nypost,Inflation rises 2.7% as Trump ramps up pressur...,https://nypost.com/2026/01/13/business/us-infl...,121
308,2026-01-13-02-38-00 +0000,wsj,White House Weighs Iran’s Nuclear-Talks Offer ...,https://www.wsj.com/world/middle-east/trump-ir...,120
182,2026-01-13-14-35-20 +0000,nyt,Trump Says He Will Impose Tariffs on Iran’s Tr...,https://www.nytimes.com/2026/01/13/world/middl...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
179,148,2026-01-13-14-47-00 +0000,wsj,Some Trump administration officials and allies...,https://www.wsj.com/politics/policy/trump-alli...
56,67,2026-01-13-19-15-03 +0000,startribune,Supreme Court seems likely to uphold state ban...,https://www.startribune.com/supreme-court-take...
350,66,2026-01-12-23-21-39 +0000,nypost,Iran to hang 26-year-old in alleged first exec...,https://nypost.com/2026/01/12/world-news/iran-...
173,62,2026-01-13-15-06-00 +0000,wsj,The Justice Department’s investigation of Fede...,https://www.wsj.com/economy/central-banking/th...
32,53,2026-01-13-20-07-43 +0000,nypost,New police dashcam video refutes claims that T...,https://nypost.com/2026/01/13/us-news/new-poli...
349,52,2026-01-12-23-31-24 +0000,nypost,"Hegseth tells defense contractors to ‘step up,...",https://nypost.com/2026/01/12/us-news/hegseth-...
14,48,2026-01-13-20-29-53 +0000,wapo,Prosecutors seek death for South Korea’s forme...,https://www.washingtonpost.com/world/2026/01/1...
1,47,2026-01-13-20-51-30 +0000,cbc,"As Carney heads to China to talk trade, Ontari...",https://www.cbc.ca/news/canada/toronto/doug-fo...
304,45,2026-01-13-03-06-52 +0000,nypost,Hakeem Jeffries calls for Kristi Noem ouster a...,https://nypost.com/2026/01/12/us-news/hakeem-j...
74,35,2026-01-13-18-32-52 +0000,nyt,"Before White House Meeting, Greenland’s Leader...",https://www.nytimes.com/2026/01/13/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
